In [ ]:
# This cell does not get executed when run with Times Square
import os
import datetime

visit_origin = 'lsstcam'
day_obs = datetime.date.today().isoformat()


In [ ]:
import os
import sys

#sched_source = 'env'
#sched_source = 'shared'
sched_source = 'devel'
match sched_source:
    case 'shared':
        if os.path.exists('/sdf/data/rubin/shared/scheduler/packages'):
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/uranography-1.2.1")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.11.0")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_sim-2.2.4")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/schedview-0.18.1.dev7+g8c79ae0")
    case 'devel':
        if os.path.exists('/sdf/data/rubin/user/neilsen/devel'):
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_scheduler")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_sim")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/schedview")
    case _:
        # Use whatever is in the kernel python environment
        pass

In [ ]:
from schedview.util import config_logging_for_reports
import logging
config_logging_for_reports(logging.ERROR)

In [ ]:
import datetime
import os
import sqlite3
import sys
from collections import defaultdict, OrderedDict
from pathlib import Path

import astropy
import astropy.units as u
import bokeh
import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting
import bokeh.transform
import cartopy
import colorcet
import healpy
import healpy as hp
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from erfa import ErfaWarning
from astropy.coordinates import SkyCoord, get_body
from astropy.time import Time
from astropy.visualization import ZScaleInterval
from IPython.display import HTML, display, Markdown, HTML
from lsst.resources import ResourcePath

In [ ]:
usdf_sim_data_dir = "/sdf/data/rubin/shared/rubin_sim_data"
if os.path.exists(usdf_sim_data_dir):
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

In [ ]:
import schedview.collect.consdb
import schedview.collect.nightreport
import schedview.collect.timeline
import rubin_scheduler
import rubin_scheduler.site_models
import rubin_scheduler.utils
import schedview.compute.maf
import schedview.compute.astro
import schedview.compute.visits
import schedview.compute.nightreport
import schedview.plot.survey_skyproj
import schedview.plot.visitmap
import schedview.plot.nightreport
import schedview.collect.visits
import rubin_sim.sim_archive
import uranography
from rubin_sim.data import get_baseline
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_sim import maf
from schedview.compute.camera import LsstCameraFootprintPerimeter
from uranography.api import Planisphere, make_zscale_linear_cmap
from schedview import DayObs
from rubin_scheduler.utils.consdb import KNOWN_INSTRUMENTS

In [ ]:
if not Path('/sdf/group/rubin/web_data/sim-data').exists():
    schedview.collect.visits.OPSIMDB_TEMPLATE = str(
        Path.home().joinpath("Data/sim-data/sims_featureScheduler_runs{sim_version}/baseline/baseline_v{sim_version}_10yrs.db")
    )

In [ ]:
# Validate the input
import re

assert visit_origin in KNOWN_INSTRUMENTS or visit_origin == 'baseline' or re.match(r"^\d+\.\d+$", visit_origin) is not None

In [ ]:
# Degraded IERS accuracy is never going to be important for these figures.

# If IERS degraded accuracy encountered, don't fail, just keep going.
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

In [ ]:
bokeh.io.output_notebook(hide_banner=True)

In [ ]:
%matplotlib inline

In [ ]:
ORIGIN_TELESCOPE = defaultdict(
    np.array(['Simonyi']).item,
    {'latiss': 'AuxTel'}
)

In [ ]:
observatory = ModelObservatory(no_sky=True)
timezone = "Chile/Continental"
telescope = ORIGIN_TELESCOPE[visit_origin]
from_opsim = visit_origin not in KNOWN_INSTRUMENTS
science_programs_to_include = ['BLOCK-365']
telescope = "AuxTel" if visit_origin.lower()=="latiss" else "Simonyi"

This report is a summary of a night of observing designed to understand scheduler behavior during the night, identify scheduler problems, and otherwise monitor scheduler performance.

In [ ]:
day_obs = DayObs.from_date(day_obs)
observatory.mjd = day_obs.mjd + 1 - observatory.location.lon.deg/360 ;# The approximate middle of the night

# Science Verification Survey Progress

### Metric maps

Each subplot represents the map of a metric a different filter, presented in a [Lambert Azimuthal Equal Area Projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection), centered at the south celestial pole.

Annotations are similar to those of the "Visit Map" above:

 - The blue backdrop represents the value of the metric.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
 - The blue line (oval) shows the plane of the Milky Way.
 - Broken red ovals show a zenith distince of 70° (airmass=2.9) at morning and evening 12° twilight.
   - The dashed portions of each red oval show the extent of what is observable at all during the night.
   - As time progress through the night, the ZD circle maintains its shape, moving counter-clockwise around the pole over the course of the night.

In [ ]:
conditions = observatory.return_conditions()

In [ ]:
visits = schedview.collect.visits.read_visits(day_obs, visit_origin, stackers = schedview.collect.visits.NIGHT_STACKERS, num_nights=10000)
visits = visits.loc[visits['science_program'].isin(science_programs_to_include), :]

### Accumulated numbers of visits

In [ ]:
fig = schedview.plot.survey_skyproj.create_metric_map_grid(
    maf.CountMetric(col='fiveSigmaDepth', metric_name="Numbers of visits"),
    visits.loc[np.isfinite(visits['fiveSigmaDepth']), :],
    observatory,
    nside=32,
)

## DDF Cadence

In [ ]:
time_window_duration=90

In [ ]:
try:
    ddf_visits = schedview.collect.visits.read_ddf_visits(day_obs, visit_origin, num_nights=time_window_duration)
    ddf_visits = ddf_visits.loc[ddf_visits['science_program'].isin(science_programs_to_include), :]
except:
    ddf_visits = []

if len(ddf_visits) > 0:
    nightly_ddf = schedview.compute.visits.accum_stats_by_target_band_night(ddf_visits)
    cadence_plots = schedview.plot.create_cadence_plot(
        nightly_ddf, day_obs.mjd - time_window_duration, day_obs.mjd
    )
    bokeh.io.show(cadence_plots)
else:
    print("No DDF visits")


The y-axis (height of the vertical bars) represents the accumulated effective exposure time, teff (as defined above) accumulated over all exposures on the field for the night, colored by filter.

## Hourglass

In [ ]:
def plot_hourglass_metric(visits, metric, constraint="", plot_dict={}, plotter=maf.plots.MonthHourglassPlot):
    seeing_bundle = maf.metric_bundles.MetricBundle(
        metric=metric,
        slicer=maf.slicers.VisitIntervalSlicer(),
        constraint=constraint,
        plot_dict=plot_dict,
        plot_funcs=[plotter(m, 2025) for m in (6, 7)],
    )
    schedview.compute.maf.compute_metric(visits, seeing_bundle)
    return seeing_bundle.plot()


### Seeing

In [ ]:
plot_hourglass_metric(visits, maf.metrics.MedianMetric("seeingFwhmEff"))

### Hour Angle

In [ ]:
plot_hourglass_metric(visits, maf.metrics.MedianMetric("HA"), plot_dict={"cmap": plt.get_cmap("coolwarm"), "color_limits": (-4.5, 4.5)})

### Band

In [ ]:
class BandUseMetric(maf.metrics.BaseMetric):
    def __init__(self, band_col="band", **kwargs):
        self.band_col = band_col
        super().__init__(col=[band_col], metric_dtype="object", **kwargs)

    def run(self, data_slice, slice_point=None):
        return data_slice[self.band_col]


In [ ]:
plot_dict = {
    "cmap": plt.get_cmap("Set1"),
    "assigned_colors": OrderedDict(
        [("u", 1), ("g", 2), ("r", 4), ("i", 7), ("z", 0), ("y", 3)]
    ),
    "legend_ncols": 1,
    "legend_loc": (1.01, 0.5),
    "legend_bbox_to_anchor": (1.01, 0.0),
    "legend": True,
}

plot_hourglass_metric(visits, BandUseMetric(), plot_dict=plot_dict, plotter=maf.plots.MonthHourglassCategoricalPlot)